<a href="https://colab.research.google.com/github/thatlq1812/DPL302m_Project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

wait for fill

In [1]:
# Google Colab library
try:
    from google.colab import drive
    import torch_xla.core.xla_model as xm
except:
    pass

In [2]:
# Python library
import os
import zipfile
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
import pickle as pkl
import yaml
import albumentations as A

from glob import glob
from tqdm import tqdm
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, as_completed

# Sklearn library
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Pytorch library
import torch

from torch import optim, nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms

# CV2 library
import cv2

c:\Users\fxlqt\.conda\envs\Py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\fxlqt\.conda\envs\Py311\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
try:
    drive.mount('/content/drive')
    if not os.path.exists("/content/data"):
        !cp "/content/drive/My Drive/Colab Notebooks/data.zip" "/content/"
        with zipfile.ZipFile("data.zip", 'r') as zip_ref:
            zip_ref.extractall("/content/")
except:
    pass

In [4]:
# Load configuration
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

# Project information
print('Project name: ', config['project']['name'])
print('Project version: ', config['project']['description'])
print('Project author: ', config['project']['author'])
print('Project version: ', config['project']['version'])

# Data location
print('Data path: ', config['data'])
data_path = config['data']['data_path']
csv_path = config['data']['csv_path']
images_path = config['data']['images_path']

# Training configuration
n_workers = multiprocessing.cpu_count()
batch_size = config['training']['batch_size']
print('N workers: ', n_workers)
print('Batch size: ', batch_size)

# Preprocessing configuration
print('Preprocessing configuration')
augmentation_ratio = config['preprocessing']['augmentation']['ratio']
resize_shape = tuple(config['preprocessing']['resize'])
print('Augmentation ratios: ', augmentation_ratio)
print('Resize shape: ', resize_shape)

# Create a dictionary for images location
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

Project name:  Cancer Detection
Project version:  DPL302m Project - Fall 2024 of Group 1
Project author:  Group 1
Project version:  1.0.0
Data path:  {'data_path': 'data/', 'csv_path': 'data/HAM10000_metadata.csv', 'images_path': 'data/images/'}
N workers:  16
Batch size:  32
Preprocessing configuration
Augmentation ratios:  1
Resize shape:  (224, 224)


In [5]:
# Data reading
data = pd.read_csv(csv_path)

In [6]:
def data_check(data):
    # Show data types
    print('Data types:')
    print(data.dtypes)
    print()

    # Show unique values for each column (skip image_id and lesion_id)
    print('Unique values for each column:')
    for col in data.columns:
        if col in ['image_id', 'lesion_id']:
            continue
        print(f"{col}: {data[col].unique()[:5]}...")
    print()

    # Count NaN values
    print('NaN values in each column:')
    nan_counts = data.isnull().sum()
    print(nan_counts[nan_counts > 0])
    print()

    # Count 'unknown' values (only for object columns)
    print('Unknown values in each column:')
    for col in data.select_dtypes(include=['object']).columns:
        unknown_count = data[data[col] == 'unknown'].shape[0]
        if unknown_count > 0:
            print(f"{col}: {unknown_count}")
    print()

    # Summary
    print("Summary of potential data issues:")
    print(f"- Columns with NaN values: {nan_counts[nan_counts > 0].index.tolist()}")
    print(f"- Columns with 'unknown' values: {[col for col in data.columns if 'unknown' in data[col].values]}")

In [7]:
data_check(data)

Data types:
lesion_id        object
image_id         object
dx               object
dx_type          object
age             float64
sex              object
localization     object
dtype: object

Unique values for each column:
dx: ['bkl' 'nv' 'df' 'mel' 'vasc']...
dx_type: ['histo' 'consensus' 'confocal' 'follow_up']...
age: [80. 75. 60. 70. 55.]...
sex: ['male' 'female' 'unknown']...
localization: ['scalp' 'ear' 'face' 'back' 'trunk']...

NaN values in each column:
age    57
dtype: int64

Unknown values in each column:
sex: 57
localization: 234

Summary of potential data issues:
- Columns with NaN values: ['age']
- Columns with 'unknown' values: ['sex', 'localization']


In [9]:
def data_cleaning(data):
    # Drop duplicates defined by 'image_id' and 'lesion_id'
    data = data.drop_duplicates(subset=['image_id'])

    # Drop values with NaN
    data = data.dropna(subset=['age', 'sex', 'localization'])
    data = data[data['age'] != 'unknown']
    data = data[data['sex'] != 'unknown']
    data = data[data['localization'] != 'unknown']

    # Convert 'age' to int
    try:
        data['age'] = data['age'].astype(int)
    except ValueError:
        print("Some 'age' values could not be converted to int.")

    # Create label encoders for each categorical column
    label_encoder_dx = LabelEncoder()
    label_encoder_dx_type = LabelEncoder()
    label_encoder_sex = LabelEncoder()
    label_encoder_localization = LabelEncoder()

    # Encode labels
    data['dx_code'] = label_encoder_dx.fit_transform(data['dx'])
    data['dx_type_code'] = label_encoder_dx_type.fit_transform(data['dx_type'])
    data['sex_code'] = label_encoder_sex.fit_transform(data['sex'])
    data['localization_code'] = label_encoder_localization.fit_transform(data['localization'])

    # Add location of images
    data['image_id'] = data['image_id'].apply(lambda x: os.path.join(images_path, x + '.jpg'))

    print('Data after cleaning:')
    print(data.head())

    data = data.reset_index(drop=True)

    return data

In [10]:
data = data_cleaning(data)
data_check(data)

Data after cleaning:
     lesion_id                      image_id   dx dx_type  age   sex  \
0  HAM_0000118  data/images/ISIC_0027419.jpg  bkl   histo   80  male   
1  HAM_0000118  data/images/ISIC_0025030.jpg  bkl   histo   80  male   
2  HAM_0002730  data/images/ISIC_0026769.jpg  bkl   histo   80  male   
3  HAM_0002730  data/images/ISIC_0025661.jpg  bkl   histo   80  male   
4  HAM_0001466  data/images/ISIC_0031633.jpg  bkl   histo   75  male   

  localization  dx_code  dx_type_code  sex_code  localization_code  
0        scalp        2             3         1                 11  
1        scalp        2             3         1                 11  
2        scalp        2             3         1                 11  
3        scalp        2             3         1                 11  
4          ear        2             3         1                  4  
Data types:
lesion_id            object
image_id             object
dx                   object
dx_type              object
age     

In [11]:
def split_data(data, test_size=0.2, random_state=42):
    # Split data into training and testing sets
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=random_state)
    print("Train data shape:", train_data.shape)
    print("Test data shape:", test_data.shape)

    return train_data, test_data

In [12]:
train_data, test_data = split_data(data, config['training']['test_size'], config['training']['random_state'])

Train data shape: (7808, 11)
Test data shape: (1953, 11)


In [13]:
def load_images_worker(batch):
    images = []
    for index, row in batch.iterrows():
        image_path = row['image_id']
        image = cv2.imread(image_path)
        if image is None:
            print('Image not found: ', image_path)
            continue
        # Preprocess image
        image = cv2.resize(image, resize_shape)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)
    return images

def load_images(data, batch_size, num_workers):
    all_images = []
    batches = [data[i:i + batch_size] for i in range(0, len(data), batch_size)]
    st = time.time()
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(load_images_worker, batch): batch for batch in batches}

        for future in tqdm(as_completed(futures), total=len(futures)):
            img_batch = future.result()
            if img_batch is not None:
                all_images.extend(img_batch)
    et = time.time()
    print('Loaded {} images in {} seconds'.format(len(all_images), et - st))
    return all_images

In [14]:
# Load train images
images = load_images(data, batch_size=batch_size, num_workers=n_workers)
train_images = load_images(train_data, batch_size=batch_size, num_workers=n_workers)
test_images = load_images(test_data, batch_size=batch_size, num_workers=n_workers)

100%|██████████| 306/306 [00:11<00:00, 27.04it/s]


Loaded 9761 images in 11.350366592407227 seconds


100%|██████████| 244/244 [00:06<00:00, 40.37it/s]


Loaded 7808 images in 6.082642316818237 seconds


100%|██████████| 62/62 [00:01<00:00, 36.43it/s]

Loaded 1953 images in 1.7812118530273438 seconds


In [15]:
def type_count(data, rate):
    lesion_type = data.groupby(data['dx']).count()
    lesion_type = lesion_type.reset_index()
    lesion_type = lesion_type[['dx', 'image_id']]
    lesion_type.columns = ['dx', 'count']
    lesion_type = lesion_type.sort_values(by='count', ascending=False)
    print(lesion_type)
    max_count = lesion_type['count'].max()
    augmentation_counts = {}
    for index, row in lesion_type.iterrows():
        augmentation_counts[row['dx']] = int(max_count / row['count'] * rate)
    return augmentation_counts

In [16]:
print("Train Data:")
augmentation_counts = type_count(train_data, config['preprocessing']['augmentation']['ratio'])
print("\nTest Data:")
_ = type_count(test_data, 0)
print("\nAugmentation Ratios:", augmentation_counts)

Train Data:
      dx  count
5     nv   5205
4    mel    876
2    bkl    849
1    bcc    404
0  akiec    265
6   vasc    120
3     df     89

Test Data:
      dx  count
5     nv   1286
2    bkl    227
4    mel    225
1    bcc    105
0  akiec     62
3     df     26
6   vasc     22

Augmentation Ratios: {'nv': 1, 'mel': 5, 'bkl': 6, 'bcc': 12, 'akiec': 19, 'vasc': 43, 'df': 58}


In [17]:
run_augmentation = True

def augment_image_worker(row, image, augmentations, augmentation_counts):
    augmented_images = []
    augmented_data = []

    lesion_type = row['dx']
    count = augmentation_counts.get(lesion_type, 0)
    if count == 0:
        return None, None

    for _ in range(count):
        if _ == 0:
            augmented_images.append(image)
            augmented_row = row.copy()
            augmented_row['image_id'] = row['image_id'] + '_original'
            augmented_data.append(augmented_row)
            continue

        augment_image = augmentations(image=image)
        augmented_images.append(augment_image['image'])
        augmented_row = row.copy()
        augmented_row['image_id'] = row['image_id'] + '_augmented_' + str(_)
        augmented_data.append(augmented_row)

    return augmented_data, augmented_images

def augment_images(data, images_array, augmentation_counts, num_workers=n_workers):
    augmentations = A.Compose([
        A.HorizontalFlip(p=0.25),
        A.VerticalFlip(p=0.25),
        A.Rotate(limit=(0, 180), p=0.25),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    ])

    print(f'Number of images: {len(images_array)}, Number of rows in data: {len(data)}')

    augmented_images = []
    augmented_data = []

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = []

        for index, row in tqdm(data.iterrows(), total=len(data), desc="Augmenting Images"):
            futures.append(
                executor.submit(augment_image_worker, row, images_array[index], augmentations, augmentation_counts)
            )

        for future in tqdm(as_completed(futures), total=len(futures), desc="Collecting Results"):
            aug_data, aug_images = future.result()
            if aug_data is not None and aug_images is not None:
                augmented_data.extend(aug_data)
                augmented_images.extend(aug_images)

    augmented_data = pd.DataFrame(augmented_data)

    return augmented_data, augmented_images


In [18]:
if run_augmentation:
    augmented_train_data, augmented_train_images = augment_images(train_data, images, augmentation_counts)
else:
    augmented_train_data, augmented_train_images = train_data.copy(), images.copy()

print('Train data shape: ', train_data.shape)
print('Augmented data shape: ', augmented_train_data.shape)
print('Augmented images length: ', len(augmented_train_images))

Number of images: 9761, Number of rows in data: 7808


Augmenting Images: 100%|██████████| 7808/7808 [00:01<00:00, 6903.10it/s]


Train data shape:  (7808, 11)
Augmented data shape:  (34884, 11)
Augmented images length:  34884


In [19]:
_ = type_count(augmented_train_data, 0)

      dx  count
5     nv   5205
3     df   5162
6   vasc   5160
2    bkl   5094
0  akiec   5035
1    bcc   4848
4    mel   4380


In [20]:
def compute_img_mean_std_worker(batch_images):
    img_w, img_h = resize_shape
    batch_imgs = []

    for image in batch_images:
        img = cv2.resize(image, (img_w, img_h))
        batch_imgs.append(img)

    return np.array(batch_imgs)

def compute_img_mean_std(augmented_images, batch_size=32, num_workers=n_workers):
    means = np.zeros(3)
    stdevs = np.zeros(3)
    total_count = 0

    # Split images into batches
    batches = [augmented_images[i:i + batch_size] for i in range(0, len(augmented_images), batch_size)]

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(compute_img_mean_std_worker, batch): batch for batch in batches}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Batches"):
            img_batch = future.result()
            if img_batch is not None:
                img_batch = img_batch.astype(np.float32) / 255.0  # Normalize

                # Compute mean and std for each channel in the batch
                for i in range(3):  # RGB channels
                    means[i] += img_batch[:, :, :, i].mean() * img_batch.shape[0]
                    stdevs[i] += (img_batch[:, :, :, i] ** 2).mean() * img_batch.shape[0]

                total_count += img_batch.shape[0]  # Update total count

    # Compute means and stds
    means /= total_count
    stdevs = np.sqrt(stdevs / total_count - means ** 2)

    return means, stdevs

In [22]:
# Return the mean and std of the images
print('Number of images: ', len(augmented_train_images))
norm_mean, norm_std = compute_img_mean_std(augmented_train_images, batch_size=batch_size, num_workers=n_workers)
print('Normed means: ', norm_mean)
print('Normed stds: ', norm_std)

# Save the mean and std into 1 file
with open('norm.pkl', 'wb') as f:
    pkl.dump({'mean': norm_mean, 'std': norm_std}, f)


Number of images:  34884


Processing Batches: 100%|██████████| 1091/1091 [00:34<00:00, 31.70it/s]


Normed means:  [0.75107293 0.56659155 0.59603678]
Normed stds:  [0.15399802 0.16798469 0.18092112]


In [23]:
train_data_droped = augmented_train_data.drop(columns=['lesion_id', 'image_id', 'dx', 'dx_type','sex','localization'])
train_data.head()
train_data_droped.head()
train_data_droped = train_data_droped.astype('float32')
train_data_droped.reset_index(drop=True, inplace=True)

In [25]:
# feature_extract is a boolean that defines if we are finetuning or feature extracting.
# If feature_extract = False, the model is finetuned and all model parameters are updated.
# If feature_extract = True, only the last layer parameters are updated, the others remain fixed.
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [29]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    # Variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18, resnet34, resnet50, resnet101
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = resize_shape[0]


    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = resize_shape[0]


    elif model_name == "densenet":
        """ Densenet121
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = resize_shape[0]

    else:
        print("Invalid model name, exiting...")
        exit()
    return model_ft, input_size

In [30]:
# resnet, vgg, densenet, densenetcustom
model_name = 'densenetcustom'
num_classes = 7
feature_extract = False
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Define the device:
try:
    device = xm.xla_device()
except:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Put the model on the device:
model = model_ft.to(device)

c:\Users\fxlqt\.conda\envs\Py311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\fxlqt\.conda\envs\Py311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda


In [31]:
# define the transformation
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(norm_mean, norm_std)])

In [32]:
# 	age	dx_code	dx_type_code	sex_code	localization_code

class HAM10000(Dataset):
    def __init__(self, df, images_array, transform=None):
        self.df = df.reset_index(drop=True)
        self.images_array = images_array  # Mảng chứa các ảnh
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Get image name
        X = self.images_array[index]
        y = torch.tensor(int(self.df['dx_code'][index]))

        if self.transform:
            X = self.transform(X)

        # return information
        additional_info = {
            'age': torch.tensor(int(self.df['age'][index])),
            'dx_type': torch.tensor(int(self.df['dx_type_code'][index])),
            'sex_code': torch.tensor(int(self.df['sex_code'][index])),
            # 'localization': torch.tensor(int(self.df['localization_code'][index]))
            # Change the additional_info_size in the CustomDenseNet121 model if you want to use 'localization'
        }

        return X, y, additional_info


In [33]:
# Define the training set using the table train_df and using our defined transitions (train_transform)
training_set = HAM10000(train_data_droped,augmented_train_images,transform=transform)
train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=n_workers)
# Same for the validation set:
validation_set = HAM10000(test_data,test_images, transform=transform)
val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=False, num_workers=n_workers)


# Lưu trữ dữ liệu để kiểm thử mà không cần load và xử lý lại ảnh, tên file có cả kích thước ảnh ( traindata64x64.pkl)
with open('traindata'+str(resize_shape[0])+'x'+str(resize_shape[1])+'.pkl', 'wb') as f:
    pkl.dump({'train_data': train_data_droped, 'train_images': augmented_train_images, 'test_data': test_data, 'test_images': test_images}, f)

with open('valdata'+str(resize_shape[0])+'x'+str(resize_shape[1])+'.pkl', 'wb') as f:
    pkl.dump({'train_data': train_data_droped, 'train_images': augmented_train_images, 'test_data': test_data, 'test_images': test_images}, f)

In [34]:
# Load lại dữ liệu
train_loader = pkl.load(open('train_loader.pkl', 'rb'))
val_loader = pkl.load(open('val_loader.pkl', 'rb'))

with open('norm.pkl', 'rb') as f:
    norm = pkl.load(f)
    norm_mean = norm['mean']
    norm_std = norm['std']

In [35]:
# we use Adam optimizer, use cross entropy loss as our loss function
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)

Train step

In [36]:
# this function is used during training process, to calculation the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [37]:
total_loss_train, total_acc_train = [],[]
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        images, labels, additional_info = data
        N = images.size(0)
        # print('image shape:',images.size(0), 'label shape',labels.size(0))
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)
        if model_name in ['densenetcustom']:
            additional_info_tensor = torch.cat([v.unsqueeze(1) for v in additional_info.values()], dim=1)
            additional_info_tensor = Variable(additional_info_tensor).to(device)
            optimizer.zero_grad()
            outputs = model(images, additional_info)
        else:
            optimizer.zero_grad()
            outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [38]:
def validate(val_loader, model, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader), total=len(val_loader)):
            images, labels, additional_info = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            if model_name in ['densenetcustom']:
                additional_info_tensor = torch.cat([v.unsqueeze(1) for v in additional_info.values()], dim=1)
                additional_info_tensor = Variable(additional_info_tensor).to(device)
                optimizer.zero_grad()
                outputs = model(images, additional_info)
            else:
                optimizer.zero_grad()
                outputs = model(images)

            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [ ]:
epoch_num = 10
best_val_acc = 0
total_loss_val, total_acc_val = [],[]
for epoch in range(1, epoch_num+1):
    print('Training on epoch {}'.format(epoch))
    loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch)
    loss_val, acc_val = validate(val_loader, model, criterion, optimizer, epoch)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        print('*****************************************************')
        print('Best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

Training on epoch 1
